### Извлекување на карактеристики од податоците на ниво на објект (CustomerID). За секој објект се гради по еден вектор од карактеристики со следниве атрибути:
   - Број на планирани посети
   - Број на извршени планирани посети
   - Број на неизвршени планирани посети
   - Број на адхок посети
   - Број на валидации (посети од контролор)
   - Средна вредност за SAPE коефициенти
   - Стандардна девијација за SAPE коефициенти
   - Средна вредност за APE коефициенти
   - Стандардна девијација за APE коефициенти
   - Квадратна разлика помеѓу средните вредности за SAPE и APE коефициентите
   - Средна вредност за FPE коефициенти
   - Стандардна девијација за FPE коефициенти
   - Број на 'ok' валидации
   - Број на 'not ok' валидации
   - Процент на 'ok' валидации
   - Процент на 'not ok' валидации


In [172]:
import pandas as pd
import numpy as np

In [173]:
df = pd.read_csv('dataset/Spenser_1_7_remove_v22_treshold.tsv',
                 delimiter='\t')

In [174]:
expert_vectors = dict()
unique_customers = set(df.CustomerID)

In [175]:
for customer in unique_customers:
    df_customer = df[df.CustomerID == customer]
    
    no_planned_visits = len(df_customer[df_customer.PerformanceEvaluationPlanned == '1'])  #број на планирани посети
    no_executed_visits = len(df_customer[df_customer.PerformanceEvaluationExecuted == '1'])  #број на извршени планирани посети
    no_not_executed_visits = no_planned_visits - no_executed_visits  #број на неизвршени планирани посети
    no_ad_hoc = len(df_customer[df_customer.PerformanceEvaluationAdHoc == '1'])  #број на адхок посети
    
    mean_sape = np.mean(df_customer.SAPECoeficient)  #средна вредност за SAPE коефициенти
    std_sape = np.std(df_customer.SAPECoeficient)  #стандардна девијација за SAPE коефициенти
    
    mean_fpe = np.mean(df_customer.FPECoeficient)  #средна вредност за FPE коефициенти
    std_fpe = np.std(df_customer.FPECoeficient)  #стандардна девијација за FPE коефициенти
    
    mean_ape = np.mean(df_customer.APECoeficient)  #средна вредност за APE коефициенти
    std_ape = np.std(df_customer.APECoeficient)  #стандардна девијација за APE коефициенти
    
    squared_err_sape_ape = (mean_sape - mean_ape)**2  # квадратна разлика помеѓу средните вредности за SAPE и APE
    
    no_ok_validations = len(df_customer[df_customer.OK == 'ok'])  #број на 'ok'валидации (OK == 'ok')
    no_not_ok_validations = len(df_customer[df_customer.NOTOK == 'not ok'])  #број на 'not ok' валидации (NOTOK == 'not ok')
    
    no_validations = len(df_customer[~np.isnan(df_customer.APECoeficient)])  #вкупен број на валидации (посети од контролор)
    
    percent_not_ok = float(no_not_ok_validations)/no_validations if no_validations != 0 else 0  #процент на 'ok' валидации
    percent_ok = float(no_ok_validations)/no_validations if no_validations != 0 else 0  #процент на 'not ok' валидации
    
    expert_vectors[customer] = [customer, no_planned_visits, no_executed_visits, no_not_executed_visits, no_ad_hoc,
                                no_validations, mean_sape, std_sape, mean_ape, std_ape, squared_err_sape_ape, 
                                mean_fpe, std_fpe, no_ok_validations, no_not_ok_validations, 
                                percent_ok, percent_not_ok]


In [171]:
cols = ['CustomerID', 'No_planned_visits', 'No_executed_visits', 'No_not_executed_visits', 'No_adhoc_visits',
        'No_validations' ,'Mean_SAPE', 'Std_dev_SAPE', 'Mean_APE', 'Std_dev_APE', 'Squared_error_SAPE_APE', 
           'Mean_FPE', 'Std_dev_FPE', 'No_ok_validations', 'No_not_ok_validations', 'Percent_ok', 'Percent_not_ok']

df_expert_vectors = pd.DataFrame(data=list(expert_vectors.values()), columns=cols)
df_expert_vectors = df_expert_vectors.sort_values(by=['CustomerID'])
df_expert_vectors.to_csv('dataset/Expert_vectors_by_customerID.tsv', sep='\t', index=None)